In [1]:
import (
    "fmt"
)

# Simple functions

In [2]:
func naiveFib(n int) int {
    if n > 1 {
        return naiveFib(n - 1) + naiveFib(n - 2)
    }
    return 1
}

In [3]:
naiveFib(20)

10946


# if, for, etc..

In [4]:
n := 10

if n > 0 {
    fmt.Println("n is positive:", n)
} else {
    fmt.Println("n is not positive:", n)
}

n is positive: 10


In [5]:
sum := 0
for i := 1; i <= n; i++ {
    sum += i
}
sum

55


# Use standard libraries

In [6]:
import (
    "fmt"
    "math"
)

In [7]:
fmt.Printf("sin(pi/3) == %f\n", math.Sin(math.Pi/3))
fmt.Printf("cos(pi/3) == %f\n", math.Cos(math.Pi/3))
fmt.Printf("sqrt(3)/2 == %f\n", math.Sqrt(3)/2)
fmt.Printf("log(e^2) == %f\n", math.Log(math.E * math.E))

sin(pi/3) == 0.866025
cos(pi/3) == 0.500000
sqrt(3)/2 == 0.866025
log(e^2) == 2.000000
21
<nil>


# Non-deterministic code

In [8]:
import (
    "math/rand"
    "time"
)

In [9]:
rand.Seed(time.Now().UnixNano())
r := rand.Int()
r

3012364053488324335


In [10]:
r % 10000

4335


In [11]:
start := time.Now()
start

2017-10-15 14:18:02.150710539 +0000 UTC m=+5.652103711


In [12]:
end := time.Now()
end

2017-10-15 14:18:02.969919145 +0000 UTC m=+6.471312244


In [13]:
fmt.Printf("end - start = %v", end.Sub(start))

end - start = 819.208533ms26
<nil>


# struct and interface

In [14]:
type person struct {
    name string
    age int
}

func (p *person) Hello() string {
    return fmt.Sprintf("Hello! Name: %s, Age: %d", p.name, p.age)
}

In [15]:
p := person{"Alice", 12}
fmt.Printf("p.name = %q\n", p.name)
fmt.Printf("p.Hello() == %q\n", p.Hello())

p.name = "Alice"
p.Hello() == "Hello! Name: Alice, Age: 12"
43
<nil>


In [16]:
type hello interface {
    Hello() string
}

func printHello(h hello) {
    if _, ok := h.(*person); ok {
        fmt.Println("h is *person")
    }
    fmt.Printf("h.Hello() == %q\n", h.Hello())
}

p := person{"Alice", 12}
printHello(&p)

h is *person
h.Hello() == "Hello! Name: Alice, Age: 12"


# channel and goroutine

In [17]:
import "fmt"

(func(){
    done := make(chan struct{})
    ch := make(chan int)
    // producer
    go func(){
        for i := 0; i < 10; i++ {
            ch <- i * i
        }
        close(ch)
    }()
    // consumer
    go func() {
        for i := range ch {
            fmt.Printf("i == %d\n", i)
        }
        close(done)
    }()
    <-done
})()

i == 0
i == 1
i == 4
i == 9
i == 16
i == 25
i == 36
i == 49
i == 64
i == 81


# panic

In [18]:
panic("fail!")

panic: fail!

goroutine 10 [running]:
runtime/debug.Stack(0xc42030f640, 0x7f3234e1a180, 0x7f320a3cdb30)
	/usr/local/go/src/runtime/debug/stack.go:24 +0xa9
main.(*handlers).HandleExecuteRequest.func3.1()
	/go/src/github.com/yunabe/lgo/cmd/lgo-internal/kernel.go:95 +0x70
panic(0x7f3234e1a180, 0x7f320a3cdb30)
	/usr/local/go/src/runtime/panic.go:491 +0x294
github.com/yunabe/lgo/cmd/runner.loadShared.func1(0xc42030f990)
	/go/src/github.com/yunabe/lgo/cmd/runner/runner.go:34 +0x108
panic(0x7f3234e1a180, 0x7f320a3cdb30)
	/usr/local/go/src/runtime/panic.go:491 +0x294
github.com/yunabe/lgo/sess7b2274696d65223a313530383037373037363434393635333537387d/exec18.lgo_init()
	/go/src/github.com/yunabe/lgo/sess7b2274696d65223a313530383037373037363434393635333537387d/exec18/src.go:4 +0x3b
github.com/yunabe/lgo/cmd/runner.loadSharedInternal(0xc4200925c0, 0x8, 0xc420378d20, 0x59)
	/go/src/github.com/yunabe/lgo/cmd/runner/runner.go:70 +0x2e9
github.com/yunabe/lgo/cmd/runner.loadShared(0x7f323533ee40, 0xc420